In [38]:
# https://medium.com/analytics-vidhya/demonstrating-calculation-of-tf-idf-from-sklearn-4f9526e7e78b

from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
import numpy as np



In [7]:
# load all the documents in the context corpus
# should be array, where each doc should be one array.
# Load the dataframe consisting of the data
df_train = pd.read_pickle("./data_frames/df_train.pkl")
df_train.head()
# columns: context_parsed (array where each sentence is a string)
# should use lemmas?

,choices,context,question,correct_answer,correct_answer_parsed,correct_answer_parsed_tok,correct_answer_parsed_stop,correct_answer_parsed_lemma,correct_answer_parsed_lemma_stop,correct_answer_loc,...,answer_location,sent_with_ans,answer_locations_text,paragraph_len,loc_frac,context_parsed,context_parsed_tok_lemma,context_parsed_tok_lemma_stop,context_parsed_tok_stop,context_parsed_tok
0,"[{'end': 224, 'extra': None, 'start': 175, 'te...",Visum för affärs- och konferensbesök\r\nOm du ...,Vad är ett visum?,ett tillstånd för att resa in och vara i ett land,ett tillstånd för att resa in och vara i ett land,"[ett, tillstånd, för, att, resa, in, och, vara...","[tillstånd, resa, in, land]","[en, tillstånd, för, att, resa, in, och, vara,...","[tillstånd, resa, in, land]",175,...,2,visum är ett tillstånd för att resa in och var...,[2],21,0.095238,"[visum för affärs- och konferensbesök, om du s...","[[visa, för, affär, och, konferensbesök], [om,...","[[visa, affär, konferensbesök], [om, skola, be...","[[visum, affärs-, konferensbesök], [om, ska, b...","[[visum, för, affärs-, och, konferensbesök], [..."
1,"[{'end': 707, 'extra': None, 'start': 681, 'te...",Eget företag\r\nEfter beslut\r\nBeslutet skick...,Vad ska du ta med när du hämtar ditt beslut?,ditt pass,ditt pass,"[ditt, pass]",[pass],"[du, pass]",[pass],156,...,2,när du ska hämta ditt beslut ska du ta med dit...,"[2, 3]",19,0.105263,"[eget företag, efter beslut beslutet skickas t...","[[egen, företag], [efter, beslut, beslut, skic...","[[egen, företag], [efter, beslut, beslut, skic...","[[eget, företag], [efter, beslut, beslutet, sk...","[[eget, företag], [efter, beslut, beslutet, sk..."
2,"[{'end': 1165, 'extra': None, 'start': 1154, '...",Utbytesdoktorander\r\nIntervju\r\nDu kommer at...,Hur länge gäller ett uppehållstillstånd för be...,ett år,ett år,"[ett, år]",[år],"[en, år]",[år],1619,...,18,så länge gäller tillståndet ett uppehållstills...,[18],25,0.720000,"[utbytesdoktorander intervju, du kommer att få...","[[utbytesdoktorand, intervju], [du, komma, att...","[[utbytesdoktorand, intervju], [du, komma, få,...","[[utbytesdoktorander, intervju], [du, kommer, ...","[[utbytesdoktorander, intervju], [du, kommer, ..."
3,"[{'end': 598, 'extra': None, 'start': 589, 'te...",Eget företag\r\nNär vi har tagit emot din ansö...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],589,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[eget företag, när vi har tagit emot din ansök...","[[egen, företag], [när, vi, ha, ta, emot, du, ...","[[egen, företag], [när, ta, emot, ansökan], [o...","[[eget, företag], [när, tagit, emot, ansökan],...","[[eget, företag], [när, vi, har, tagit, emot, ..."
4,"[{'end': 1932, 'extra': None, 'start': 1924, '...",Flytta tillbaka till Sverige\r\nSå ansöker du\...,Vad är ett uppehållstillståndskort?,ett bevis,ett bevis,"[ett, bevis]",[bevis],"[en, bevis]",[bevis],673,...,8,kortet är ett bevis på att du har tillstånd at...,[8],21,0.380952,"[flytta tillbaka till sverige, så ansöker du d...","[[flytta, tillbaka, till, Sverige], [så, ansök...","[[flytta, tillbaka, Sverige], [så, ansöka, du,...","[[flytta, tillbaka, sverige], [så, ansöker, du...","[[flytta, tillbaka, till, sverige], [så, ansök..."


In [8]:
def combine_docs(df, col):
    docs = []
    for index, row in df.iterrows():
        context = row[col]
        if isinstance(context[0], list):
            sents = []
            for sent in context:
                sent_str = ' '.join(sent)
                sents.append(sent_str)
            context = sents
        doc = ' '.join(context)
        docs.append(doc)
    return docs

In [9]:
tfidf_transformer = TfidfTransformer()
cv = CountVectorizer()

In [10]:
def create_tf_df(docs):
    word_count_vector = cv.fit_transform(docs)
    tf = pd.DataFrame(word_count_vector.toarray(), columns=cv.get_feature_names())
    return word_count_vector, tf


In [18]:
def create_idf_df(word_count_vector):
    X = tfidf_transformer.fit_transform(word_count_vector)
    idf = pd.DataFrame({'feature_name':cv.get_feature_names(), 'idf_weights':tfidf_transformer.idf_})
    return X, idf

In [19]:
docs = combine_docs(df_train, 'context_parsed')
word_count_vector, tf = create_tf_df(docs)
X, idf = create_idf_df(word_count_vector)
tf_idf = pd.DataFrame(X.toarray() ,columns=cv.get_feature_names())
print(tf_idf)

          000  0000  010  011  0128   03   05  0630   07  0730  ...  \
0    0.051548   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
1    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
2    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
3    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
4    0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
..        ...   ...  ...  ...   ...  ...  ...   ...  ...   ...  ...   
957  0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
958  0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
959  0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
960  0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   
961  0.000000   0.0  0.0  0.0   0.0  0.0  0.0   0.0  0.0   0.0  ...   

     övervägande  övning  övningar  övningarna  övningsanläggningen  \
0            0.0     0.0       0.0         0.0                  0.0   
1    

/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [33]:
docs = combine_docs(df_train, 'context_parsed_tok_lemma_stop')
word_count_vector_lemma_stop, tf_lemma_stop = create_tf_df(docs)
X_lemma_stop, idf = create_idf_df(word_count_vector_lemma_stop)
tf_idf_lemma_stop = pd.DataFrame(X_lemma_stop.toarray() ,columns=cv.get_feature_names())
print(tf_idf_lemma_stop)

     000  0000  010  011  0128  01454  03  05  06  06975  ...  övervakning  \
0      1     0    0    0     0      0   0   0   0      0  ...            0   
1      0     0    0    0     0      0   0   0   0      0  ...            0   
2      0     0    0    0     0      0   0   0   0      0  ...            0   
3      0     0    0    0     0      0   0   0   0      0  ...            0   
4      0     0    0    0     0      0   0   0   0      0  ...            0   
..   ...   ...  ...  ...   ...    ...  ..  ..  ..    ...  ...          ...   
957    0     0    0    0     0      0   0   0   0      0  ...            2   
958    0     0    0    0     0      0   0   0   0      0  ...            2   
959    0     0    0    0     0      0   0   0   0      0  ...            0   
960    0     0    0    0     0      0   0   0   0      0  ...            0   
961    0     0    0    0     0      0   0   0   0      0  ...            0   

     övervakningsarbete  överväga  övervägande  övning  övnings

/Users/filippakarrfelt/opt/anaconda3/envs/dp/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [14]:
# save tf-idf values
tf_idf.to_pickle("./data_frames/tf-idf/df_tf_idf.pkl")
tf_idf_lemma_stop.to_pickle("./data_frames/tf-idf/df_tf_idf_lemma_stop.pkl")

# create tf-idf vectors based on the bigger corpus idf values

In [28]:
# load and compute the tf-idf values based on the bigger corpus
idf_sv_df = pd.read_csv("../idf_sv.csv")
idf_sv_df.head()


,word,idf
0,medverka,5.365197
1,årskurs,5.770662
2,som,0.695488
3,arbetsmiljö­arbetet,6.463809
4,personalens,6.463809


In [31]:
# create idf vec
def create_idf_sv_corpus(idf, idf_sv_df):
    idf_mod = idf.copy()
    for index, row in idf_mod.iterrows():
        word = row['feature_name']
        matching_row = idf_sv_df.loc[idf_sv_df['word'] == word]
        if not matching_row.empty:
            row['idf_weights'] = matching_row.iloc[0]['idf'] # replace the idf score by the one from the bigger corpus
    return idf_mod

In [32]:
idf_sv_corpus = create_idf_sv_corpus(idf, idf_sv_df)

In [44]:
# compute tf_idf by iterating over each row in the
def tf_idf_from_sv_idf(tf, idf):
    tf_idf = tf.copy()
    tf_idf = tf_idf.mul(idf['idf_weights'].values, axis=1)

    # normalize the tf_idf values
    sqrt_vec = np.sqrt(tf_idf.pow(2).sum(axis=1))
    tf_idf_norm = tf_idf.div(sqrt_vec, axis=0)
    return tf_idf_norm


        
tf_idf_sv_corpus = tf_idf_from_sv_idf(tf_lemma_stop, idf_sv_corpus)

In [45]:
# save the tf-idf values from the larger corpus
tf_idf_sv_corpus.to_pickle("./data_frames/tf-idf/df_tf_idf_lemma_stop_sv_corpus.pkl")
